In [8]:
# https://huggingface.co/docs/trl/en/sft_trainer#format-your-input-prompts
# https://huggingface.co/docs/trl/en/sft_trainer#supervised-fine-tuning-trainer
# https://huggingface.co/docs/trl/en/sft_trainer#dataset-format-support
# https://huggingface.co/docs/transformers/main/en/chat_templating#templates-for-chat-models
# def format_reddit_post(sample):
#     answer = sample["answer\r"]
#     question = sample["question"]
#     conversation = {
#         "messages": [
#             {"role": "system", "content": "You help answer questions for UT Austin students"},
#             {"role": "user", "content": question.replace("\n", "").replace("\r", "") },
#             {"role": "assistant", "content": answer.replace("\n", "").replace("\r", "")}

#         ]
#     }

#     return conversation

from datasets import load_dataset, Dataset
import pandas as pd

# load the processed dataset
base_path = "Bevo-Bud-The-GPT/model-src"
# train_data = load_dataset("csv", data_files=f"{base_path}/train-v1.csv")
train_data = load_dataset("json", data_files=f"{base_path}/train-v1.json", split='train')

# train_df = pd.read_csv(f"{base_path}/train-v1.csv", lineterminator='\n')
# train_df.dropna(inplace=True)

# dataset_hf = Dataset.from_pandas(train_df)
# # dataset_hf['answer\r']
# train_data = dataset_hf.map(format_reddit_post, batched=False)

In [81]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import setup_chat_format
base_model = "distilbert/distilgpt2"

model = AutoModelForCausalLM.from_pretrained(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True, max_length=1024)

model, tokenizer = setup_chat_format(model, tokenizer)

# tokenizer.pad_token = tokenizer.eos_token
version = "1.2"
print("Saving as verion: ", version)


Saving as verion:  1.2


In [24]:
from trl import SFTTrainer 
from peft import LoraConfig, get_peft_model

training_args = {
    # "per_device_train_batch_size": 2,
    "per_device_eval_batch_size": 4,
    # "gradient_accumulation_steps": 4,
    "num_train_epochs": 3,
    # "logging_steps": 1,
    # "save_steps": 100,
    "seed": 42,
    "output_dir": f"{base_path}/Bevo-Bud",
    # "push_to_hub": True,
    "overwrite_output_dir": True
}

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
# model = get_peft_model(model, peft_config)

training_args = TrainingArguments(**training_args)
trainer = SFTTrainer(
            model,
            train_dataset=train_data,
            tokenizer=tokenizer,
            packing=True,
            peft_config=peft_config,
            args=training_args,
            
        )

trainer.train()

Step,Training Loss


TrainOutput(global_step=420, training_loss=10.769466145833333, metrics={'train_runtime': 14528.6956, 'train_samples_per_second': 0.23, 'train_steps_per_second': 0.029, 'total_flos': 879309241712640.0, 'train_loss': 10.769466145833333, 'epoch': 3.0})

In [33]:
trainer.save_model(f"{base_path}/Bevo-Bud")

/usr/local/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


### Testing

In [59]:
from transformers import pipeline, Conversation
from peft import PeftConfig, PeftModel

tokenizer = AutoTokenizer.from_pretrained(f"{base_path}/Bevo-Bud")
config = PeftConfig.from_pretrained(f"{base_path}/Bevo-Bud")
# base_model = AutoModelForCausalLM.from_pretrained(base_model)

generator = pipeline(model=f"{base_path}/Bevo-Bud-Model", tokenizer=tokenizer)
# generator = pipeline('text-generation', model=f"{base_path}/Bevo-Budv{version}", tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [79]:
# pip install peft transformers
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
# from accelerate import infer_auto_device_map, init_empty_weights
model = AutoModelForCausalLM.from_pretrained(
    "distilbert/distilgpt2",
    #load_in_4bit=True,
)
adapters_name = f"{base_path}/Bevo-Bud"
model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()


TypeError: PeftModel.from_pretrained() missing 1 required positional argument: 'model_id'

In [ ]:

peft_model_id = f"{base_path}/Bevo-Bud"

config = PeftConfig.from_pretrained(peft_model_id)

model1 = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model1 = PeftModel.from_pretrained(model, peft_model_id)

In [75]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
# from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = f"{base_path}/Bevo-Bud"

config = PeftConfig.from_pretrained(peft_model_id)

'distilbert/distilgpt2'

In [92]:
from transformers import pipeline, Conversation
from peft import PeftConfig, PeftModel

tokenizer = AutoTokenizer.from_pretrained(f"{base_path}/Bevo-Bud")
config = PeftConfig.from_pretrained(f"{base_path}/Bevo-Bud")
# model = AutoModelForCausalLM.from_pretrained(f"{base_path}/Bevo-Bud")
# base_model = AutoModelForCausalLM.from_pretrained(base_model)

generator = pipeline(model=f"{base_path}/Bevo-Bud-Model", tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/usr/local/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


RuntimeError: Error(s) in loading state_dict for GPT2LMHeadModel:
	size mismatch for transformer.wte.weight: copying a param with shape torch.Size([50259, 768]) from checkpoint, the shape in current model is torch.Size([50257, 768]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([50259, 768]) from checkpoint, the shape in current model is torch.Size([50257, 768]).

In [94]:

pipe2 = pipeline("text-generation", model=base_model)
pipe2("What are good places to study? at UT")
# generator = pipeline('text-generation', model=f"{base_path}/Bevo-Budv{version}", tokenizer=tokenizer)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "What are good places to study? at UT&T International (Texas TX), an international university that specializes in international students' education.\n\n\n\n\n\nTo see full schedule, visit www.utsc.edu."}]

In [83]:

messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot that helps answer questions related to the University of Texas at Austin (UT Austin)",
    },
    {
        "role": "user",
        "content": "Can I still graduate if my GPA is below the minimum?"
    }
]


In [91]:
# print(model.transformer.wte.weight.shape[0])
# print(len(tokenizer))

50259
50259


In [87]:
generator(messages, max_new_tokens=128)

IndexError: index out of range in self

In [52]:
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

<|im_start|>system
You are a friendly chatbot that helps answer questions related to the University of Texas at Austin (UT Austin)<|im_end|>
<|im_start|>user
Can I still graduate if my GPA is below the minimum?<|im_end|>
<|im_start|>assistant



In [53]:
outputs = model.generate(tokenized_chat, max_new_tokens=128) 
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a friendly chatbot that helps answer questions related to the University of Texas at Austin (UT Austin)<|im_end|>
<|im_start|>user
Can I still graduate if my GPA is below the minimum?<|im_end|>
<|im_start|>assistant



































































































































IndexError: index out of range in self

## Uploading Model To Hub

In [82]:
from huggingface_hub import notebook_login
notebook_login()
trainer.push_to_hub(f"Bevo-Bud")

events.out.tfevents.1713985198.82d7450fbbdd.5942.0:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1713985235.82d7450fbbdd.5942.1:   0%|          | 0.00/5.57k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Kelechie/tmp_trainer/commit/a5daaa92c8176af9a9d307f73b97b613e83db901', commit_message='Bevo-Budv1.0', commit_description='', oid='a5daaa92c8176af9a9d307f73b97b613e83db901', pr_url=None, pr_revision=None, pr_num=None)